In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from joblib import load

In [2]:
cred = credentials.Certificate('./firebase-sdk.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
# Function to fetch a user's data from Firestore
def fetch_user_data(user_id):
    doc_ref = db.collection('users').document(user_id)
    doc = doc_ref.get()
    if doc.exists:
        return doc.to_dict()
    else:
        print("No such document!")
        return None

# Function to preprocess user's data
def preprocess_user_data(user_data):
    # Assuming user_data is a dictionary containing the user's features
    df = pd.DataFrame([user_data])  # Convert dict to DataFrame
    
    # Apply the same preprocessing as your training data
    # Load label encoder and transform 'Country of Birth'
    le_country = load('label_encoder.joblib')
    df['encoded_country'] = le_country.transform(df['CountryofBirth'])
    
    # Ensure all preprocessing steps match training, including dropping unused columns
    processed_df = df.drop(['Name', 'CountryofBirth'], axis=1)
    
    # Return preprocessed user data ready for prediction
    return processed_df

# Load your trained Decision Tree model
DT_modelcv = load('classifier.joblib')

# Predict function for a user
def predict_tbpred(user_id):
    user_data = fetch_user_data(user_id)
    if user_data:
        processed_data = preprocess_user_data(user_data)
        prediction = DT_modelcv.predict(processed_data)
        return prediction[0]
    else:
        return "User data not found"

# Example usage
user_id = 'Id2CXsv9CuIUMTTGDLgg'  # Replace with actual user ID
prediction = predict_tbpred(user_id)
print(f"Predicted tbpred for user {user_id}: {prediction}")

Predicted tbpred for user Id2CXsv9CuIUMTTGDLgg: 1


C:\Users\alice\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
